## Input

In [ ]:
# input sample
# first converation
{
    "conversion_type": "first",
    "ocr_json": {
            "ocr_result": [],
            "img_string": "THE ROBERT",
            "img_structure": []
    },
    "user_input":{},
    "conversation_history":[]
}

# continued converation
{
    "conversion_type": "continued",
    "ocr_json": {},
    "user_input":{
        "XXXXXXXX"
    },
    "conversation_history": [
        {"role": "system", "content": input_system},  # system prompt
        {"role": "user", "content": input_text},
        {"role": "user", "content": input_doc} # first input doc that generated from OCR
    ]
}

## Chat with ChatGPT

In [ ]:
from openai import OpenAI
from dotenv import load_dotenv
import os

load_dotenv() # load API key

client = OpenAI(
  api_key=os.environ.get("OPENAI_API_KEY"),
)

In [ ]:
def chat_with_gpt(messages):
    completion = client.chat.completions.create(
        model="gpt-4",  # or "gpt-3.5-turbo"
        messages=messages,
        temperature=0
    )
    return completion.choices[0].message.content

### The very first conversation with ChatGPT
- input: document output from OCR

### json input for testing

In [62]:
import json

# f = open('../sample_data/sample.json')
# data = json.load(f)
# f.close()
# f = open('../sample_data/BudgetRequestSummary.json')
f = open('../sample_data/California-Standard-Residential-Lease-Agreement-Sophia.json')
# f = open('../sample_data/ID_Card_0.json')
# f = open('../sample_data/ss-5_nina_printed.json')
data = json.load(f)
f.close()

data = data["pages"][0]

In [63]:
input_doc = '<document>'+data['img_string']+'</document>'
input_doc

# list
# input_docs = []
# for page in data['pages']:
#     input_docs.append("<doc>"+page['img_string']+"</doc>")

# string
# input_docs = ""
# for page in data['pages']:
#     input_docs += "<doc>"+page['img_string']+"</doc>"

# input_docs

"<document>CALIFORNIA RESIDENTIAL LEASE/RENTAL AGREEMENT\nLANDLORD  Prime Properties Rental Management\nTENANT(S) Olivia Johnson\nNoah Davis\nPROPERTY ADDRESS:  123 Maple Street\nSpringfield, IL 62704\n1. RENTAL AMOUNT: Beginning _08/29 2024 TENANT agrees to pay LANDLORD\nthe sum of $ 2999.99 per month in advance on the 1 day of each calendar month.\nSaid rental payment shall be delivered by TENANT to LANDLORD or his designated agent to the\nfollowing location: 123 Maple Street, Springfield, IL 62704  Rent must be\nactually received by LANDLORD, or designated agent, in order to be considered in compliance with\nthe terms of this agreement.\n2. TERM: The premises are leased on the following lease term: (please check one item only)\n Month-to-Month\n(or)\n Until _08/29 ,20 30\n3. SECURITY DEPOSITS: TENANT shall deposit with landlord the sum of $ 2999.99 as a\nsecurity deposit to secure TENANT'S faithful performance of the terms of this lease. The security.\ndeposit shall not exceed two t

### system config

In [82]:
input_system = """You are now a strict sensitive information protection officer. You must protect customer privacy and ensure that sensitive information in documents is not leaked. Here are some categories of personal data: Name, Phone Number, ID Number, Social Security Number (SSN), Email Address, Address, Date of Birth, Gender, Nationality, Passport Number, Bank Account Number, Driver's License Number, Occupation, Health Information, Marital Status, Income or Wage. These types of data (including but not limited to) are usually considered sensitive and need to be properly protected to prevent unauthorized access and use."""

positive_examples = """For example, here is a sample document that contains sensitive information:

<document>Dear Loan Department Manager, My name is Zhang Xiaoming, currently residing at Guangfu South Road, Xinyi District, Taipei City. I am hereby submitting a loan application to your esteemed bank. The purpose of this letter is to request a loan to support my personal financial needs. I plan to use the loan funds for home renovation to enhance the comfort of my residence. As a loan applicant, I am willing to comply with all the terms and conditions set by your bank. I understand that the loan application may require the submission of additional documentation, and I am ready to cooperate and complete these procedures. Below is a summary of my personal profile and financial status: Personal Profile: I am an IT professional with over five years of experience working at a renowned technology company. I have a good credit record and repayment capability. Income Situation: My primary source of income is my monthly salary, which provides a stable income sufficient to support the repayment plan. Financial Status: My financial status is stable, and I have several years of financial records that can demonstrate my repayment ability. I sincerely hope that your bank will approve my loan application, allowing me to achieve the aforementioned financial goals. I am willing to provide further information or documents at any time to assist in the review of my application. Sincerely, Zhang Xiaoming July 2, 2024</document>

You need to list out the sensitive information like this:
"- Name: Zhang Xiaoming
- Address: Guangfu South Road, Xinyi District, Taipei City"
No explanation is needed.
"""

# positive_examples = """For example, here is a sample document that contains sensitive information:

# <document>THE ROBERT A.WELCH FOUNDATION\n2010 Bank of the Southwest Building\nHouston,Texas 77002\nBUDGET REQUEST SUMMARY\nYEAR (AS APPLICABLE\nMay 11966 May ,19 May I,19 Total\nthrough through\nApr.30.19 Apr.30.19\n1.Personnel $11228.00 $ $11228.00\n2.Permanent Scientific Equipment. S\n3.Expendable Scientific Items &Services 840.00 $ $ 840.00\n4.Other Expense $ 975.00 $ $ 975.00\n5.TOTAL Exclusive of Overhead $13.043.00 $ $13.043.00\n6.Overhead 1957.00 $ $ 51957.00\n7.TOTAL AMOUNT of Proposed Budget .. $15000.00 $15.000.00\nNames of Principal InvestigatorsJohn E.Kilpat rick\nInstitution William Marsh Rice University\nTHE SPACE BELOW IS FOR USE BY THE FOUNDATION\nGrant Period 19 to 19 Grant No\nDate\nApproved Not Approved\nDirector of Research\nScientific Advisory Board\nBoard of Trustees\nGrantee and Institution Notified\nRemarks:\nSource:https://www.industrydocuments.ucsf.edu/docs/zxfk0226\n</document>

# You need to list out the sensitive information like this:
# "- Name: John E.Kilpat rick"
# No explanation is needed.
# """

duplicate_info = """If a document contains duplicate sensitive information appearing in multiple different positions, keep all of them without merging.
For example, you might see a paragraph in the document that shows: '405 Hilgard Avenue Los' 'Angeles, California, 90095'
And another paragraph shows: '405 Hilgard Avenue, Los Angeles, California, 90095' Please keep both.

You need to list out the sensitive information like this:
"- Address: 405 Hilgard Avenue, Los Angeles, California, 90095
- Address: 405 Hilgard Avenue
- Address: Los Angeles, California, 90095
"
"""

input_text = """Here is a document. Identify the text containing sensitive information and list it with bullet points, like this: "- Name: Amy, - SSN: 123456789." No explanation needed:"""


In [83]:
# input_text = """
# identify the sensitive information:
# THE ROBERT A.WELCH FOUNDATION\n2010 Bank of the Southwest Building\nHouston,Texas 77002\nBUDGET REQUEST SUMMARY\nYEAR (AS APPLICABLE\nMay 11966 May ,19 May I,19 Total\nthrough through\nApr.30.19 Apr.30.19\n1.Personnel $11228.00 $ $11228.00\n2.Permanent Scientific Equipment. S\n3.Expendable Scientific Items &Services 840.00 $ $ 840.00\n4.Other Expense $ 975.00 $ $ 975.00\n5.TOTAL Exclusive of Overhead $13.043.00 $ $13.043.00\n6.Overhead 1957.00 $ $ 51957.00\n7.TOTAL AMOUNT of Proposed Budget .. $15000.00 $15.000.00\nNames of Principal InvestigatorsJohn E.Kilpat rick\nInstitution William Marsh Rice University\nTHE SPACE BELOW IS FOR USE BY THE FOUNDATION\nGrant Period 19 to 19 Grant No\nDate\nApproved Not Approved\nDirector of Research\nScientific Advisory Board\nBoard of Trustees\nGrantee and Institution Notified\nRemarks:\nSource:https://www.industrydocuments.ucsf.edu/docs/zxfk0226\n
# """

In [97]:
# initialize the conversation history
conversation_history = [
    {"role": "system", "content": input_system},  # system prompt
    {"role": "system", "content": positive_examples},
    {"role": "system", "content": duplicate_info},
    {"role": "user", "content": input_text},
    {"role": "user", "content": input_doc} # first input doc that generated from OCR
]

In [98]:
response_from_gpt = chat_with_gpt(conversation_history)
print(response_from_gpt)

- Name: Prime Properties Rental Management
- Name: Olivia Johnson
- Name: Noah Davis
- Address: 123 Maple Street, Springfield, IL 62704
- Name: James Taylor
- Name: Isabella Miller


### Multi-turn dialogue with ChatGPT

In [99]:
# number of turns
i = 0

# main conversation
while True:

    # get user input
    user_input = input("You: ") # TBD with backend
    
    # check if the user wants to exit
    if user_input.lower() in ['exit', 'quit', 'bye', 'ok', 'thanks', 'nice', 'great']: # TBD with backend
        print("============================")
        print("Goodbye :)")
        break
    

    # customize user input to eventual prompts
    # ======================================= #
    #             customize here              #
    # ======================================= #


    # add user input prompts to conversation history
    conversation_history.append({"role": "user", "content": user_input})
    
    # get response from ChatGPT
    response_from_gpt = chat_with_gpt(conversation_history)
    i += 1
    
    # add ChatGPT response to conversation history
    conversation_history.append({"role": "assistant", "content": response_from_gpt}) # role is assistant
    
    # print the response
    print("============================")
    print(f"Turn {i}:")
    print("User input:", user_input)
    print(response_from_gpt) # .split('\n')

Turn 1:
User input: also add two-line format address aditionally
- Name: Olivia Johnson
- Name: Noah Davis
- Address: 123 Maple Street, Springfield, IL 62704
- Address: 123 Maple Street
- Address: Springfield, IL 62704
- Name: James Taylor
- Name: Isabella Miller
Goodbye :)


## Post processing
mapping the value & bounding box

In [12]:
masks = []

for line in response_from_gpt.split('\n'):
    words = line.split('- ')[-1].split(': ')[-1]
    print(words)

    for i in range(len(data['ocr_result'])):
        text = data['ocr_result'][i]['text']
        start_index = text.find(words)
        
        if start_index != -1:
            if start_index!=0:
                print(f"{start_index=}")
                print(f"{len(words)=}, {len(text)=}")
                end_index = start_index + len(words)
                
                # the start and the end proportion of words in text
                start_proportion = start_index / len(text)
                end_proportion = end_index / len(text)
                print(f"{start_proportion=}, {end_proportion=}")

                # calculate new bounding box based on the proportion
                bbox = data['ocr_result'][i]['bbox']
                bbox_width = bbox[2] - bbox[0]
                new_bbox_left = round(bbox[0] + start_proportion * bbox_width, 1)
                new_bbox_right = round(bbox[0] + end_proportion * bbox_width, 1)

                print(bbox)
                print([new_bbox_left, bbox[1]], [new_bbox_right, bbox[3]])
                masks.append({"top_left": [new_bbox_left, bbox[1]],
                            "bottom_right": [new_bbox_right, bbox[3]]})


John E.Kilpat rick
[103.0, 756.0, 649.0, 788.0]
[452.4, 756.0] [649.0, 788.0]
2010 Bank of the Southwest Building, Houston,Texas 77002
Principal Investigators
[103.0, 756.0, 649.0, 788.0]
[201.3, 756.0] [452.4, 788.0]
William Marsh Rice University
[105.0, 849.0, 542.0, 874.0]
[232.9, 849.0] [542.0, 874.0]


1. John E. Kilpatrick
{'top_left': [753.02582248324, 429.9212346964], 'bottom_right': [782.7093872214, 652.6092418772]}

2. William Marsh Rice University
{'top_left': [844.196771322, 202.9915511885], 'bottom_right': [876.0005906843733, 544.4464955322]}

1. John E. Kilpatrick
{'top_left': [0.4471649777216389, 0.3609750081414287], 'bottom_right': [0.46479179763743733, 0.5479506648842744]}

2. William Marsh Rice University
{'top_left': [0.5013044960344484, 0.17043791031776695], 'bottom_right': [0.5201903745156611, 0.45713391732346353]}

In [18]:
# text = 'Name(s) of Principal Investigators) -John E.Kilpat rick*asdf'
# # words = 'John E. Kilpatrick'
# words = 'John E.Kilpat rick'

# bbox = [
#                 152.0, # 0
#                 1010.0, # 1
#                 949.0, # 2
#                 1052.0 # 3
#             ]

# text.find(words)

# start_index = text.find(words)
# if start_index != -1:
#     end_index = start_index + len(words)
    
#     # 計算 words 在 text 中的起始和結束比例
#     start_proportion = start_index / len(text)
#     end_proportion = end_index / len(text)

#     # 根據比例計算 bounding box 的新位置
#     # bbox = data['ocr_result'][i]['bbox']
#     bbox_width = bbox[2] - bbox[0]
#     new_bbox_left = round(bbox[0] + start_proportion * bbox_width, 1)
#     new_bbox_right = round(bbox[0] + end_proportion * bbox_width, 1)

#     print(bbox)
#     masks.append({"top_left": [new_bbox_left, bbox[3]],
#                     "bottom_right": [new_bbox_right, bbox[1]]})


#     print({"top_left": [new_bbox_left, bbox[3]],
#                     "bottom_right": [new_bbox_right, bbox[1]]})

[152.0, 1010.0, 949.0, 1052.0]
{'top_left': [643.4833333333333, 1052.0], 'bottom_right': [882.5833333333333, 1010.0]}


## Output
- `conversion_type`: ["first", "continued"]
- output mask bbox

In [10]:
# output sample
response = [
    {
        "conversion_type": "first", # or "continued",
        "conversation_history": {
            {"role": "system", "content": input_system},  # system prompt
            {"role": "user", "content": input_text},
            {"role": "user", "content": input_doc}, # first input doc that generated from OCR
            {"role": "user", "content": user_input}, # first user input
            {"role": "assistant", "content": response_from_gpt}, # gpt output
            {"role": "user", "content": user_input}, # second user input
            {"role": "assistant", "content": response_from_gpt},
            {"role": "user", "content": user_input}, # third user input
            {"role": "assistant", "content": response_from_gpt}
        },
        "masks": [
                {"top_left": [152.0, 1010.0], "bottom_right": [1052.0, 1052.0]},
                {"top_left": [147.0, 1137.0], "bottom_right": [1187.0, 1187.0]}
            ]
    }
]